In [2]:
import pandas as pd
import json

In [3]:
df = pd.read_json('../data/user_responses.json')

In [4]:
df.recommendations.head()

0    {'CF': ['990', '586', '797', '390', '49', '268...
1    {'CF': ['762', '394', '73', '635', '528', '288...
2    {'CF': ['44', '558', '394', '77', '726', '107'...
3    {'CF': ['968', '168', '495', '888', '198', '97...
4    {'CF': ['120', '733', '2', '599', '944', '869'...
Name: recommendations, dtype: object

In [5]:
json_struct = json.loads(df.to_json(orient="records"))    
df_flat = pd.json_normalize(json_struct) #use pd.io.json


In [6]:
df_keep = df_flat[['userId', 'formAnswers.Generation.question_1','formAnswers.Generation.question_2','formAnswers.Generation.question_3','recommendations.CF',
'recommendations.VCS']]

In [7]:
df_keep = df_keep.rename(columns={
    "formAnswers.Generation.question_1": "question_1",
    "formAnswers.Generation.question_2": "question_2",
    "formAnswers.Generation.question_3": "question_3", 
    "recommendations.CF": "CF",
    "recommendations.VCS": "VCS"
    })

In [8]:
df_keep.head()

,userId,question_1,question_2,question_3,CF,VCS
0,75674,[A],[Strongly Agree],[Strongly Agree],"[990, 586, 797, 390, 49, 268, 510, 894, 203, 9...","[797, 990, 586, 49, 510, 970, 327, 894, 250, 2..."
1,39849,[A],[Strongly Agree],[Strongly Agree],"[762, 394, 73, 635, 528, 288, 383, 410, 438, 5...","[73, 383, 394, 410, 762, 288, 563, 528, 228, 2..."
2,22986,[A],[Agree],[Strongly Agree],"[44, 558, 394, 77, 726, 107, 315, 686, 35, 384...","[558, 77, 394, 315, 686, 107, 35, 384, 726, 76..."
3,78878,[A],[Agree],[Disagree],"[968, 168, 495, 888, 198, 978, 513, 552, 450, ...","[888, 552, 450, 68, 714, 329, 518, 978, 513, 4..."
4,63931,[A],[Agree],[Neither agree nor disagree],"[120, 733, 2, 599, 944, 869, 755, 910, 196, 79...","[120, 733, 944, 869, 196, 755, 779, 790, 649, ..."


In [9]:
df_keep['weight'] = 1

In [10]:
df_keep['question_1'] = df_keep['question_1'].apply(lambda x: x[0])
df_keep['question_2'] = df_keep['question_2'].apply(lambda x: x[0])
df_keep['question_3'] = df_keep['question_3'].apply(lambda x: x[0])

In [11]:
df_keep.head()

,userId,question_1,question_2,question_3,CF,VCS,weight
0,75674,A,Strongly Agree,Strongly Agree,"[990, 586, 797, 390, 49, 268, 510, 894, 203, 9...","[797, 990, 586, 49, 510, 970, 327, 894, 250, 2...",1
1,39849,A,Strongly Agree,Strongly Agree,"[762, 394, 73, 635, 528, 288, 383, 410, 438, 5...","[73, 383, 394, 410, 762, 288, 563, 528, 228, 2...",1
2,22986,A,Agree,Strongly Agree,"[44, 558, 394, 77, 726, 107, 315, 686, 35, 384...","[558, 77, 394, 315, 686, 107, 35, 384, 726, 76...",1
3,78878,A,Agree,Disagree,"[968, 168, 495, 888, 198, 978, 513, 552, 450, ...","[888, 552, 450, 68, 714, 329, 518, 978, 513, 4...",1
4,63931,A,Agree,Neither agree nor disagree,"[120, 733, 2, 599, 944, 869, 755, 910, 196, 79...","[120, 733, 944, 869, 196, 755, 779, 790, 649, ...",1


In [12]:
def reweight(df):
    translate_negative = {'Strongly Agree':0.3, 'Agree':0.4, 'Neither agree nor disagree':0.5, 'Disagree':0.6, 'Strongly Disagree':0.7}
    translate_positive = {'Strongly Agree':0.7, 'Agree':0.6, 'Neither agree nor disagree':0.4, 'Disagree':0.3, 'Strongly Disagree':0.3}
    return [(df.weight * translate_negative[df.question_2] *  translate_positive[df.question_3])]

df_keep2 = df_keep
df_keep2['new_weighted'] = df_keep.apply(reweight, axis=1,result_type='expand')

,userId,question_1,question_2,question_3,CF,VCS,weight,new_weighted
0,75674,A,Strongly Agree,Strongly Agree,"[990, 586, 797, 390, 49, 268, 510, 894, 203, 9...","[797, 990, 586, 49, 510, 970, 327, 894, 250, 2...",1,0.21
1,39849,A,Strongly Agree,Strongly Agree,"[762, 394, 73, 635, 528, 288, 383, 410, 438, 5...","[73, 383, 394, 410, 762, 288, 563, 528, 228, 2...",1,0.21
2,22986,A,Agree,Strongly Agree,"[44, 558, 394, 77, 726, 107, 315, 686, 35, 384...","[558, 77, 394, 315, 686, 107, 35, 384, 726, 76...",1,0.28
3,78878,A,Agree,Disagree,"[968, 168, 495, 888, 198, 978, 513, 552, 450, ...","[888, 552, 450, 68, 714, 329, 518, 978, 513, 4...",1,0.12
4,63931,A,Agree,Neither agree nor disagree,"[120, 733, 2, 599, 944, 869, 755, 910, 196, 79...","[120, 733, 944, 869, 196, 755, 779, 790, 649, ...",1,0.16


In [14]:
df_keep2.head()

,userId,question_1,question_2,question_3,CF,VCS,weight,new_weighted
0,75674,A,Strongly Agree,Strongly Agree,"[990, 586, 797, 390, 49, 268, 510, 894, 203, 9...","[797, 990, 586, 49, 510, 970, 327, 894, 250, 2...",1,0.21
1,39849,A,Strongly Agree,Strongly Agree,"[762, 394, 73, 635, 528, 288, 383, 410, 438, 5...","[73, 383, 394, 410, 762, 288, 563, 528, 228, 2...",1,0.21
2,22986,A,Agree,Strongly Agree,"[44, 558, 394, 77, 726, 107, 315, 686, 35, 384...","[558, 77, 394, 315, 686, 107, 35, 384, 726, 76...",1,0.28
3,78878,A,Agree,Disagree,"[968, 168, 495, 888, 198, 978, 513, 552, 450, ...","[888, 552, 450, 68, 714, 329, 518, 978, 513, 4...",1,0.12
4,63931,A,Agree,Neither agree nor disagree,"[120, 733, 2, 599, 944, 869, 755, 910, 196, 79...","[120, 733, 944, 869, 196, 755, 779, 790, 649, ...",1,0.16


### Absoulte Average

In [18]:
df_keep2.groupby('question_1')['new_weighted'].sum()/df_keep2.shape[0]

question_1
A    0.093333
B    0.103333
Name: new_weighted, dtype: float64

In [27]:
data = df_keep2.groupby('question_2')['question_2'].describe()
data.index.name = None
print(data[['freq']].to_latex())

\begin{tabular}{ll}
\toprule
{} & freq \\
\midrule
Agree                      &   12 \\
Disagree                   &    3 \\
Neither agree nor disagree &    6 \\
Strongly Agree             &   15 \\
\bottomrule
\end{tabular}



/var/folders/_b/xsxs0vzs0dg6t2wd_jx_dqx00000gn/T/ipykernel_23008/2637567958.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(data[['freq']].to_latex())


### Relative Average

In [35]:
dataA = df_keep2.groupby('question_1')
dataB = df_keep2.groupby('question_1')

In [45]:
# Get n of A
n = df_keep2.groupby('question_1').count().userId.A
dataA = df_keep2.groupby('question_1')['new_weighted'].sum()/n
dataA

question_1
A    0.186667
B    0.206667
Name: new_weighted, dtype: float64

In [46]:
# Get n of A
n = df_keep2.groupby('question_1').count().userId.B
dataB = df_keep2.groupby('question_1')['new_weighted'].sum()/n
dataB

question_1
A    0.186667
B    0.206667
Name: new_weighted, dtype: float64

In [15]:
data = df_keep2.groupby('question_3')['question_3'].describe()
data.index.name = None
# print(data[['freq']].to_latex())
data

,count,unique,top,freq
Agree,13,1,Agree,13
Disagree,5,1,Disagree,5
Neither agree nor disagree,13,1,Neither agree nor disagree,13
Strongly Agree,5,1,Strongly Agree,5


In [16]:
submission = {"1854": "5", "1880": "4", "1924": "5", "2002": "2", "2055": "1","7011": "5", "6002": "4", "5967": "5", "5003": "4", "50": "1"}
subdf = pd.DataFrame(submission, columns=['id', 'rated'])

In [28]:
df_keep2.head()

,userId,question_1,question_2,question_3,CF,VCS,weight,new_weighted
0,75674,A,Strongly Agree,Strongly Agree,"[990, 586, 797, 390, 49, 268, 510, 894, 203, 9...","[797, 990, 586, 49, 510, 970, 327, 894, 250, 2...",1,0.005833
1,39849,A,Strongly Agree,Strongly Agree,"[762, 394, 73, 635, 528, 288, 383, 410, 438, 5...","[73, 383, 394, 410, 762, 288, 563, 528, 228, 2...",1,0.005833
2,22986,A,Agree,Strongly Agree,"[44, 558, 394, 77, 726, 107, 315, 686, 35, 384...","[558, 77, 394, 315, 686, 107, 35, 384, 726, 76...",1,0.007778
3,78878,A,Agree,Disagree,"[968, 168, 495, 888, 198, 978, 513, 552, 450, ...","[888, 552, 450, 68, 714, 329, 518, 978, 513, 4...",1,0.003333
4,63931,A,Agree,Neither agree nor disagree,"[120, 733, 2, 599, 944, 869, 755, 910, 196, 79...","[120, 733, 944, 869, 196, 755, 779, 790, 649, ...",1,0.004444


In [52]:
data3 = df_keep2.groupby('question_1')['question_2','question_3'].describe()
data3

/var/folders/_b/xsxs0vzs0dg6t2wd_jx_dqx00000gn/T/ipykernel_8078/1263436607.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data3 = df_keep2.groupby('question_1')['question_2','question_3'].describe()


question_2                             question_3         \
                count unique             top freq      count unique   
question_1                                                            
A                  18      3           Agree    8         18      4   
B                  18      4  Strongly Agree    8         18      4   

                                             
                                   top freq  
question_1                                   
A           Neither agree nor disagree    6  
B           Neither agree nor disagree    7

In [38]:
data2 = df_keep2.groupby('question_1')['new_weighted'].describe()

In [40]:
data2.index.name = None

In [44]:
data2

,count,mean,std,min,25%,50%,75%,max
A,18.0,0.005185,0.001617,0.0025,0.004236,0.005000,0.006458,0.008333
B,18.0,0.005741,0.002477,0.0025,0.003611,0.005278,0.006667,0.010000


In [48]:
l = pd.pivot_table(data2, columns=['A','B'])

In [51]:
print(l.to_latex())

\begin{tabular}{lrr}
\toprule
{} &          A &          B \\
\midrule
25\%   &   0.004236 &   0.003611 \\
50\%   &   0.005000 &   0.005278 \\
75\%   &   0.006458 &   0.006667 \\
count &  18.000000 &  18.000000 \\
max   &   0.008333 &   0.010000 \\
mean  &   0.005185 &   0.005741 \\
min   &   0.002500 &   0.002500 \\
std   &   0.001617 &   0.002477 \\
\bottomrule
\end{tabular}



/var/folders/_b/xsxs0vzs0dg6t2wd_jx_dqx00000gn/T/ipykernel_8078/1919725570.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(l.to_latex())
